In [1]:
library(readxl)
library(tidyverse)
library(rstatix) # for Dunn's test
library(reshape2)
library(car) #package necessary to run anova and specify type III errors
setwd('/Users/alexis/IEHS Dropbox/Rager Lab/Alexis_Payton/Experiments/6. Elise\'s Project/6.1. Adjusted P Values')

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some




In [2]:
# Import data table as data frame and make Subject IDs row labels.
MediatorData <- data.frame(read_excel("Input Data/2021_06_29 IS Project Mediators.xlsx"))
DemographicData <- read.csv("Input Data/2021_06_15 IS Project Demographics.csv")

head(MediatorData)
head(DemographicData)

,Subject_ID,Device,IL1B,IL6,IL8,TNFa,MMP2,MMP9,MPO,NE,⋯,MIP1a,MIP1B,TARC,bFGF,Flt1,PIGF,Tie2,VEGFC,VEGFD,VEGF
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CA_001,NS/NV,22.160324,0.000000,443.36981,2.323988,0.00,149694.090,1244612.30,60007.688,⋯,0.000000,0.00000,0.000000,2.6072384,34.57635,0.6930062,0,0.000,0.00000,36.773
2,CA_004,NS/NV,11.900966,0.402586,434.97369,1.493960,0.00,49942.969,189463.93,128626.299,⋯,0.000000,0.00000,0.000000,3.0455254,19.72112,0.0000000,0,0.000,0.00000,202.390
3,CA_005,NS/NV,59.725809,44.571240,2534.21002,5.286530,134533.87,419045.656,1634252.49,827759.205,⋯,7.587209,17.09581,1.861665,32.6238660,163.24190,2.8183238,0,421.748,296.05379,753.540
4,CA_006,NS/NV,4.195704,0.000000,93.43214,0.000000,0.00,7314.684,62439.25,1685.938,⋯,0.000000,0.00000,0.000000,1.6172160,18.84291,0.0000000,0,0.000,0.00000,13.115
5,CA_008,NS/NV,108.562329,10.401007,1230.45746,2.531193,1846.45,207838.128,2207070.70,151337.112,⋯,7.587209,16.10194,0.000000,0.6185674,18.84291,0.0000000,0,0.000,0.00000,747.850
6,CA_009,4th Gen,251.857446,66.319425,11015.01379,23.168781,10899.47,2007683.962,5784581.26,699896.405,⋯,28.942287,19.93613,1.607421,119.2757727,172.59075,0.0000000,0,1257.001,34.26977,302.490


,Subject.ID,Device,Sex,Race,Hispanic.,Age,BMI,Serum.Cotinine..ng.mL..ELISA
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>
1,CA 001,NS/NV,F,W,NO,22,27.3,NA
2,CA 004,NS/NV,F,W,NO,22,20.5,NA
3,CA 005,NS/NV,F,W,NO,21,25.3,0
4,CA 006,NS/NV,M,W,YES,21,27.9,NA
5,CA 008,NS/NV,M,B,NO,27,31.8,NA
6,CA 009,4th Gen,M,W,NO,19,20.1,NA


In [3]:
MediatorData_Devices <- data.frame(MediatorData$Device, row.names = MediatorData$Subject_ID)
colnames(MediatorData_Devices) <- "Device"
MediatorData <- data.frame(MediatorData[ , 3:47], row.names = MediatorData$Subject_ID)

# Filter out mediators that were detected in fewer than 25 subjects (~25% of the cohort, since we have four groups).
MediatorData <- MediatorData[, which(as.numeric(colSums(MediatorData != 0)) > 25)]

# Impute zeroes as the sqrt of the lowest value in that column.
MediatorData <- as.data.frame(apply(MediatorData[ , 1:43], 2, function(x) replace(x, x == 0, sqrt(min(x[x>0])))))

# Put column names in alphabetical order
MediatorData <- MediatorData[,order(names(MediatorData))]

# Add groups back in to data frame
MediatorData <- transform(merge(MediatorData_Devices, MediatorData, by=0 ,all=TRUE), row.names=Row.names, Row.names=NULL)

# Identify factors that will be the columns in your table and specify the order you want them to appear.
MediatorData$Device <- factor(MediatorData$Device, 
                              levels=c("NS/NV", "SM", "3rd Gen", "4th Gen"), 
                              labels=c("NS/NV", "Smoker", "3rd Gen", "4th Gen"))
                                    
head(MediatorData)

,Device,Albumin,CRP,Eotaxin,Eotaxin3,Flt1,GMCSF,IFNg,IL10,IL12p40,⋯,TNFa,Tie2,Uteroglobin,VEGF,VEGFC,VEGFD,bFGF,dsDNA,sICAM1,sVCAM1
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CA_001,NS/NV,6050.353,1.624004,3.063601,0.1497121,34.57635,2.8684272,0.35019,0.4383821,0.3769871,⋯,2.3239884,4.391904,86614.15,36.773,6.249917,2.904066,2.6072384,732641.62,59.76169,484.5604
CA_004,NS/NV,5676.526,1.624004,3.063601,0.1497121,19.72112,0.0750228,0.35019,0.4908329,3.5529171,⋯,1.4939595,4.391904,142763.90,202.390,6.249917,2.904066,3.0455254,484265.39,156.55747,278.8425
CA_005,NS/NV,63070.352,25.175647,3.063601,0.1497121,163.24190,1.3472141,0.35019,0.5434500,45.1865518,⋯,5.2865301,4.391904,27853666.44,753.540,421.748017,296.053788,32.6238660,2966355.10,5083.30967,980.6836
CA_006,NS/NV,103.468,1.624004,70.163381,0.1497121,18.84291,0.7748752,0.35019,0.3133017,0.1421193,⋯,0.4151426,4.391904,388568.51,13.115,6.249917,2.904066,1.6172160,29327.11,85.61421,484.5604
CA_008,NS/NV,9476.822,488.797071,3.063601,0.1497121,18.84291,2.6303678,0.35019,0.6385332,1.5336766,⋯,2.5311930,4.391904,153297.26,747.850,6.249917,2.904066,0.6185674,1507876.51,1133.60912,880.4278
CA_009,4th Gen,427479.975,2.637389,139.728304,0.1497121,172.59075,0.0750228,0.35019,0.6703242,335.5747095,⋯,23.1687805,4.391904,832761.07,302.490,1257.001344,34.269769,119.2757727,6341595.70,500.19741,2626.2062


# Table E2

P values from Kruskal-Wallis tests for Table E2. 

In [4]:
cytokines = colnames(MediatorData)[2:44]

cytokine_kruskal_test_pvalues = data.frame()
for(i in 1:length(cytokines)){
    kruskal_test = kruskal.test(as.formula(paste0(cytokines[i],"~", "Device")), data = MediatorData)
    kruskal_test_values = cbind("Cytokine" = cytokines[i], "P Value" = kruskal_test$p.value)
    cytokine_kruskal_test_pvalues = rbind(cytokine_kruskal_test_pvalues, kruskal_test_values)
}

#adding adjusted p values
cytokine_kruskal_test_pvalues$`P Adjust` = p.adjust(cytokine_kruskal_test_pvalues$`P Value`, method = "BH")

head(cytokine_kruskal_test_pvalues)

,Cytokine,P Value,P Adjust
,<chr>,<chr>,<dbl>
1,Albumin,0.00333630191813482,0.017932623
2,CRP,0.000448900965999647,0.004086496
3,Eotaxin,0.555883828243207,0.620095199
4,Eotaxin3,0.609657727165508,0.639397128
5,Flt1,0.000110510514372353,0.001583984
6,GMCSF,0.0241009840932995,0.060961313


# Table E3

P values from ANOVA/ANCOVA for Table E3.

In [14]:
# combining both datasets
all_data = full_join(MediatorData, DemographicData)[-c(45, 48, 50, 51)] %>%
    mutate(Device = factor(ifelse(Device == "Smoker", "SM", Device)), Sex = factor(Sex), 
           Race = factor(ifelse(Race != "W", "NW", Race)))
head(all_data)

Joining, by = "Device"


,Device,Albumin,CRP,Eotaxin,Eotaxin3,Flt1,GMCSF,IFNg,IL10,IL12p40,⋯,VEGF,VEGFC,VEGFD,bFGF,dsDNA,sICAM1,sVCAM1,Sex,Race,Age
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<int>
1,NS/NV,6050.353,1.624004,3.063601,0.1497121,34.57635,2.868427,0.35019,0.4383821,0.3769871,⋯,36.773,6.249917,2.904066,2.607238,732641.6,59.76169,484.5604,F,W,22
2,NS/NV,6050.353,1.624004,3.063601,0.1497121,34.57635,2.868427,0.35019,0.4383821,0.3769871,⋯,36.773,6.249917,2.904066,2.607238,732641.6,59.76169,484.5604,F,W,22
3,NS/NV,6050.353,1.624004,3.063601,0.1497121,34.57635,2.868427,0.35019,0.4383821,0.3769871,⋯,36.773,6.249917,2.904066,2.607238,732641.6,59.76169,484.5604,F,W,21
4,NS/NV,6050.353,1.624004,3.063601,0.1497121,34.57635,2.868427,0.35019,0.4383821,0.3769871,⋯,36.773,6.249917,2.904066,2.607238,732641.6,59.76169,484.5604,M,W,21
5,NS/NV,6050.353,1.624004,3.063601,0.1497121,34.57635,2.868427,0.35019,0.4383821,0.3769871,⋯,36.773,6.249917,2.904066,2.607238,732641.6,59.76169,484.5604,M,NW,27
6,NS/NV,6050.353,1.624004,3.063601,0.1497121,34.57635,2.868427,0.35019,0.4383821,0.3769871,⋯,36.773,6.249917,2.904066,2.607238,732641.6,59.76169,484.5604,F,W,27


In [15]:
# scaling data
# I SCALED, BUT I THINK YOU TRANSFORMED SO THIS PREPARATION STEP LIKELY NEEDS TO BE CHANGED
all_data[2:44] = lapply(all_data[2:44], scale, 2)
head(all_data)

,Device,Albumin,CRP,Eotaxin,Eotaxin3,Flt1,GMCSF,IFNg,IL10,IL12p40,⋯,VEGF,VEGFC,VEGFD,bFGF,dsDNA,sICAM1,sVCAM1,Sex,Race,Age
,<fct>,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>",⋯,"<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>","<dbl[,1]>",<fct>,<fct>,<int>
1,NS/NV,0.01706856,-0.0001822607,0.004949431,-0.1154564,0.1117626,0.2923906,-0.006325856,-1.156973,-0.00838527,⋯,0.02434447,0.01006584,0.00164984,0.01862786,0.1647137,0.0009096682,0.07564241,F,W,22
2,NS/NV,0.01706856,-0.0001822607,0.004949431,-0.1154564,0.1117626,0.2923906,-0.006325856,-1.156973,-0.00838527,⋯,0.02434447,0.01006584,0.00164984,0.01862786,0.1647137,0.0009096682,0.07564241,F,W,22
3,NS/NV,0.01706856,-0.0001822607,0.004949431,-0.1154564,0.1117626,0.2923906,-0.006325856,-1.156973,-0.00838527,⋯,0.02434447,0.01006584,0.00164984,0.01862786,0.1647137,0.0009096682,0.07564241,F,W,21
4,NS/NV,0.01706856,-0.0001822607,0.004949431,-0.1154564,0.1117626,0.2923906,-0.006325856,-1.156973,-0.00838527,⋯,0.02434447,0.01006584,0.00164984,0.01862786,0.1647137,0.0009096682,0.07564241,M,W,21
5,NS/NV,0.01706856,-0.0001822607,0.004949431,-0.1154564,0.1117626,0.2923906,-0.006325856,-1.156973,-0.00838527,⋯,0.02434447,0.01006584,0.00164984,0.01862786,0.1647137,0.0009096682,0.07564241,M,NW,27
6,NS/NV,0.01706856,-0.0001822607,0.004949431,-0.1154564,0.1117626,0.2923906,-0.006325856,-1.156973,-0.00838527,⋯,0.02434447,0.01006584,0.00164984,0.01862786,0.1647137,0.0009096682,0.07564241,F,W,27


In [16]:
contrasts(all_data$Device)

,4th Gen,NS/NV,SM
3rd Gen,0,0,0
4th Gen,1,0,0
NS/NV,0,1,0
SM,0,0,1


In [22]:
cytokine_anova = data.frame()
covariates = c("Sex", "Race", "Age")

for(i in 1:length(cytokines)){
    for(j in 1:length(covariates)){
        anova_results = aov(as.formula(paste0(cytokines[i] + ,"~", "Device")), data = all_data)
        anova_type3 = Anova(anova_results, type = 'III')

        cytokine_summary = anova_type3[2,3:4]
        p_value = cytokine_summary$`Pr(>F)`

        cytokine_anova_values = cbind("Cytokine" = cytokines[i], "Model" = "Anova", "P Value" = p_value)
        cytokine_anova = rbind(cytokine_anova, cytokine_anova_values)
    }
}

#adding adjusted p values and model
cytokine_anova$`P Adjust` = p.adjust(cytokine_anova$`P Value`, method = "BH")

head(cytokine_anova)

,Cytokine,Model,P Value,P Adjust
,<chr>,<chr>,<chr>,<dbl>
1,Albumin,Anova,0.464510791706394,4.645108e-01
2,CRP,Anova,1.41218426637209e-17,3.195996e-17
3,Eotaxin,Anova,0.00116008506476241,1.279068e-03
4,Eotaxin3,Anova,2.17438737138878e-15,4.452317e-15
5,Flt1,Anova,1.30585961462576e-22,3.743464e-22
6,GMCSF,Anova,0.0627023474223681,6.419526e-02


In [ ]:
#adjusting for sex age and race
cytokine_ancova = data.frame()

for(i in 1:length(cytokines)){
    anova_results = aov(as.formula(paste0(cytokines[i],"~", "Device")), data = all_data)
    anova_type3 = Anova(anova_results, type = 'III')
    
    cytokine_summary = anova_type3[2,3:4]
    p_value = cytokine_summary$`Pr(>F)`

    cytokine_ancova_values = cbind("Cytokine" = cytokines[i], "Model" = "Anova", "P Value" = p_value)
    cytokine_ancova = rbind(cytokine_ancova, cytokine_ancova_values)
    
}

#adding adjusted p values and model
cytokine_ancova$`P Adjust` = p.adjust(cytokine_ancova$`P Value`, method = "BH")

head(cytokine_ancova)